# Road Following and Collision Avoidance

In this notebook we will use the models we trained on both Road Following and Collision Avoidance so that our robot will be able to follow the track and avoid collisions at the same time!

Before starting let's first upload our trained models *best_steering_model_xy_trt.pth* and *best_collision_model.pth* into this file directory

## TensorRT

In [ ]:
import torch
device = torch.device('cuda')

Load the TRT optimized model for both Road Following and Collision Avoidance by executing the cell below

In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth'))

model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('best_collision_model_trt.pth')) # anti collision model trained for one object as blocked and street as free

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera. You should be pretty familiar with this by now. 

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera=Camera.instance(width=224, height=224,fps=10)

In [ ]:
image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

We'll also create our robot instance which we'll need to drive the motors.

In [ ]:
from jetbot import Robot

robot = Robot()

Now, let's define sliders to control the JetBot, **Blocked slider** to see the probability that the robot is blocked, **Stoptime slider** to select maually the time-for-stop when object has been detected and lastly **Blocked threshold** slider to manually set the threshold for when to stop depending on the blocked probability
.

1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
2. Steering Gain Control (steering_gain_sloder): If you see JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

In [8]:
#Road Following
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.10, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.23, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

#Collision Avoidance
blocked_slider = ipywidgets.FloatSlider(description='blocked', min=0.0, max=1.0)
stoptime_slider= ipywidgets.IntSlider(description='time to stop', min=1, max=100, step=1, value=10) 
blocked_threshold= ipywidgets.FloatSlider(description='blocked threshold', min=0, max=1.0, step=0.01, value=0.8)

#display(image_widget)

display(ipywidgets.HBox([blocked_slider, blocked_threshold, stoptime_slider]))

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.1, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.23, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. Compute the approximate steering value
4. Control the motors using proportional / derivative control (PD)

In [ ]:
import time
import math

angle = 0.0
angle_last = 0.0
go_on=1
count_stops=0

def execute(change):
    global angle, angle_last, go_on, count_stops
    
    image = change['new']
    image_preprocessed = preprocess(image)
    
    #Collsion Avoidance
    y_collision = model_trt_collision(image_preprocessed)
    
    #We apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y_collison = F.softmax(y_collison) #F.softmax(y_collison, dim=1) 
    
    prob_blocked = float(y_collison.flatten()[0])
    blocked_slider.value = prob_blocked
    
    if go_on==1:
        if prob_blocked > block_threshold.value:
            speed_slider.value=0.0
            go_on=2
        else:
            #start of road following
            count_stops=0
            xy = model_trt(image_preprocessed).detach().float().cpu().numpy().flatten()
            x = xy[0]
            y = (0.5 - xy[1]) / 2.0
    else:
        count_stops += 1
        if count_stops<stop_time:
            x=0.0 #set steering zero
            y=0.0 #set steering zero
            speed_gain_slider.value=0 #set speed zero
        else:
            go_on=1
            count_stops=0
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0)*0.88, 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

> WARNING: This code will move the robot!! Please make sure your robot has clearance.  The Road Following and Collision Avoidance should work, but the neural
> network is only as good as the data it's trained on!

In [ ]:
camera.observe(execute, names='value')

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. Try placing your robot on the track and see whether it is able to follow the track and try placing an object which you have trained in the collision avoidance and see whether the robot stops. 

If you want to stop the Jetbot and unattach the camera execute the code below.

In [ ]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

### Conclusion

That's it for this live demo! Hopefully you had some fun seeing your robot move smoothly across the track as well as avoid collisions!

If your robot wasn't following the road very well or wasn't avoiding collisions properly, try to spot where it fails. The beauty is that we can collect more data for these failure scenarios and the JetBot should get even better :)